In [ ]:
import numpy as np
import matplotlib.pyplot as pl
from scipy.stats.stats import pearsonr


from create_dictionaries import create_dictionaries
import copy


from itertools import cycle
from sklearn.linear_model import Lasso, ElasticNet, lasso_path, enet_path
from sklearn.linear_model import LassoCV, ElasticNetCV, LassoLarsCV, LassoLarsIC

from sklearn.model_selection import KFold

import json

import sys

from IPython.utils.io import Tee

In [ ]:
activations_name='weights0904-2d-30Hz-newinputs-dataaugmentation-30epochs'
#activations_folder="../activations/Scores_Ramus/weights_2020-04-07"
n_cells=128


metrics_folder='./corpus_ramus/Files/'

saveTxtFile=True #save best cells in text file

l1_ratio=0.2 #for ElasticNet
#NB further parameters: conservative, type of cells

saveCoeffs=True #save regression coeffs to json file
normalizeFeatures=False

In [ ]:
activations_folder=f'../activations/Scores_Ramus/{activations_name}'

In [ ]:
dfiles, d_match, datay, d_act, d_lang, D, dmetrics = create_dictionaries(activations_folder=activations_folder,
                                                           metrics_folder=metrics_folder)


### Metrics

In [ ]:
metrique_keys=['deltV', 'deltC', 'propV', 'varco_V', 'varco_C', 'nPVI_V', 'rPVI_C']
list_files=list(d_match.keys())

metric_arrays={}
for metrique in metrique_keys:
    l=[]
    for file in list_files:
        l.append(dmetrics[file][metrique])
    metric_arrays[metrique]=np.array(l)

Figures metrics (NB: they differ slightly from Ramus' figures as not every audio file was available, especially for English)

In [ ]:
metrique_tuples=[('propV', 'deltC'), ('propV', 'deltV'), ('deltV', 'deltC'), ('nPVI_V', 'rPVI_C')]
plt=pl

for tup in metrique_tuples:
    plt.figure(figsize=(7,5))
    metrique_x, metrique_y = tup
    xarr=metric_arrays[metrique_x]
    yarr=metric_arrays[metrique_y]
    
    
    if metrique_x in ['deltC', 'deltV']:
        shift_x=0.001
    elif metrique_x in ['propV', 'nPVI_V', 'rPVI_C']:
        shift_x=0.1

    if metrique_y in ['deltC', 'deltV']:
        shift_y=0.001
    elif metrique_y in ['propV', 'nPVI_V', 'rPVI_C']:
        shift_y=0.1
    for lang in D:
        xarr_lang=[]
        yarr_lang=[]
        for j, file in enumerate(list_files):
            if d_lang[file] == lang:
                xarr_lang.append(xarr[j])
                yarr_lang.append(yarr[j])
        xarr_lang=np.array(xarr_lang)
        yarr_lang=np.array(yarr_lang)
        std=np.std(xarr_lang)
        xsterr=std/np.sqrt(len(xarr_lang))
        std=np.std(yarr_lang)
        ysterr=std/(np.sqrt(len(yarr_lang)))
        x,y=np.mean(xarr_lang),np.mean(yarr_lang)
        plt.errorbar(x,y,label=lang,fmt=".k",xerr=xsterr,yerr=ysterr,capsize=2)
        plt.text(x+shift_x,y+shift_y,lang)

    plt.title(f"Distribution des langues selon {metrique_x}, {metrique_y}")
    plt.xlabel(metrique_x)
    plt.ylabel(metrique_y)
    plt.show()

### Pearson coefficients

NB: Pearson coefficients are signed (contrary to original version where it was the absolute value)

In [ ]:

pearson={}

for metrique in metrique_keys:
    pearson[metrique]={'lstm_1':{'cell_states':[],'outputs':[]},'lstm_2':{'cell_states':[],'outputs':[]}}

for lstm in ['lstm_1','lstm_2']:
    for celltype in ['outputs','cell_states']:
        for i in range(n_cells): #i passe sur chacune des cellules
            y=[] 
            for file in list_files:
                audio=d_match[file].split('.')[0]
                y.append(datay[audio][lstm][celltype][i])
            y=np.array(y).astype(np.float)
            
            for metrique in metrique_keys:
                x=metric_arrays[metrique]
                pearson[metrique][lstm][celltype].append(pearsonr(y,x)[0])


Meilleures cellules

In [ ]:

##Meilleures cellules
'''
d1={}
for metrique in list(pearson.keys()):
    d1[metrique]=[]
    for lstm in list(pearson[metrique].keys()):
        for celltype in list(pearson[metrique][lstm].keys()):
            d1[metrique]=d1[metrique]+pearson[metrique][lstm][celltype]
'''

if saveTxtFile:
    #f=open(f'./corpus_ramus/best_cells_{activations_name}.txt', 'w')
    #orig_stdout = sys.stdout
    #sys.stdout=f
    tee=Tee(f'./corpus_ramus/best_cells_{activations_name}.txt')

for metrique in list(pearson.keys()):
    pearson2 = copy.deepcopy(pearson)
    print(metrique)
    max_ref_previous=np.inf
    for i in range(5):
        max_ref=-np.inf
        for lstm in list(pearson2[metrique].keys()):
            for celltype in list(pearson2[metrique][lstm].keys()):
                for ind, act in enumerate(pearson2[metrique][lstm][celltype]):
                    if max_ref<np.abs(act) and np.abs(act)<max_ref_previous:
                        ind_max=lstm, celltype, ind
                        max_ref=np.abs(act)
        lstm, celltype, ind=ind_max                
        text=lstm+", "+celltype+", "+str(ind)+'\t\t'
        print(text,max_ref)
        max_ref_previous=max_ref
    print("\n")


if saveTxtFile:
    tee.close()
    #f.close()
    #sys.stdout=orig_stdout

Figures

### Scatter plots

In [ ]:
metrique_inds={'deltV':0, 'deltC':1, 'propV':2}
c_lang={"en":'C0',"pol":'C1',"du":'C2',"fr":'C3',"esp":'C4',"it":'C5',"cat":'C6',"ja":'C7'}


### (Regularized) linear regressions

In [ ]:
#reference layer
layer='lstm_2'
celltype='outputs'


#metric_coeffs={'propV':0.05, 'deltC':20, 'deltV':20, 'nPVI_V':0.02, 'rPVI_C':0.15} #the metrics are first multiplied by these coeffs (y in [0, 1] range)
metric_coeffs=dict([(metrique, 1./metric_arrays[metrique].std()) for metrique in metrique_keys])

Matrix of activations / metric vectors

In [ ]:
X=np.zeros((n_cells, len(d_match)))

metric_arrays={}
for metrique in metrique_keys:
    l=[]
    for j, file in enumerate(list_files):
        l.append(dmetrics[file][metrique])
        audio=d_match[file].split('.')[0]
        X[:, j] = np.array(datay[audio][layer][celltype]).astype(np.float)
    metric_arrays[metrique]=np.array(l)

AIC/BIC criterion / CV

In [ ]:
#normalize data?
X2=np.copy(X)
XT=X2.T
if normalizeFeatures:
    XT -= np.mean(XT, axis=0)

    XT /= XT.std(axis=0)

k_fold=7
shuffling=False
EPSILON= 1e-4

random_state = 1 if shuffling else None
CVsplitter=KFold(n_splits=k_fold, shuffle=shuffling, random_state=random_state)

metric_alphaCV_lasso = {}
metric_alphaCV_enet = {}

plt=pl
for metrique in metric_arrays:
    try:
        y=metric_coeffs[metrique]*metric_arrays[metrique]
    except KeyError as e:
        print(f'{metrique} not in metric_coeffs, lasso not performed')
        continue
    '''
    model_bic = LassoLarsIC(criterion='bic')
    model_bic.fit(XT, y)
    
    alpha_bic_ = model_bic.alpha_

    model_aic = LassoLarsIC(criterion='aic')
    model_aic.fit(XT, y)
    alpha_aic_ = model_aic.alpha_


    def plot_ic_criterion(model, name, color):
        criterion_ = model.criterion_
        plt.semilogx(model.alphas_ + EPSILON, criterion_, '--', color=color,
                     linewidth=3, label='%s criterion' % name)
        plt.axvline(model.alpha_ + EPSILON, color=color, linewidth=3,
                    label='alpha: %s estimate' % name)
        plt.xlabel(r'$\alpha$')
        plt.ylabel('criterion')


    plt.figure()
    plot_ic_criterion(model_aic, 'AIC', 'b')
    plot_ic_criterion(model_bic, 'BIC', 'r')
    plt.legend()
    plt.title(f'Information-criterion for model selection {metrique}')
    
    '''

    # #############################################################################
    # LassoCV: coordinate descent

    # Compute paths
    
    eps = 5e-2  # the smaller it is the longer is the path
    
    print("Computing regularization path using the coordinate descent lasso...")
    model_lasso = LassoCV(cv=CVsplitter, eps=eps, fit_intercept=True).fit(XT, y)
    print("Computing regularization path using the elastic net...")
    model_enet = ElasticNetCV(cv=CVsplitter, l1_ratio=l1_ratio, eps=eps, fit_intercept=True).fit(XT, y)

    metric_alphaCV_lasso[metrique] = model_lasso.alpha_
    metric_alphaCV_enet[metrique] = model_enet.alpha_
    
    
    # Display results
    plt.figure(figsize=(10,6))
    plt.subplot(1, 2, 1)
    ymin, ymax = 0.02, 0.1
    plt.semilogx(model_lasso.alphas_ + EPSILON, model_lasso.mse_path_, ':')
    plt.plot(model_lasso.alphas_ + EPSILON, model_lasso.mse_path_.mean(axis=-1), 'k',
             label='Average across the folds', linewidth=2)
    plt.axvline(model_lasso.alpha_ + EPSILON, linestyle='--', color='k',
                label='alpha: CV estimate')

    plt.legend()

    plt.xlabel(r'$\alpha$')
    plt.ylabel('Mean square error')
    plt.title(f'Lasso {metrique}')
    plt.axis('tight')
    #plt.ylim(ymin, ymax)
    
    
    
    plt.subplot(1, 2, 2)
    plt.semilogx(model_enet.alphas_ + EPSILON, model_enet.mse_path_, ':')
    plt.plot(model_enet.alphas_ + EPSILON, model_enet.mse_path_.mean(axis=-1), 'k',
             label='Average across the folds', linewidth=2)
    plt.axvline(model_enet.alpha_ + EPSILON, linestyle='--', color='k',
                label='alpha: CV estimate')

    plt.legend()

    plt.xlabel(r'$\alpha$')
    plt.ylabel('Mean square error')
    plt.title(f'ElasticNet {metrique}')
    plt.axis('tight')
    #plt.ylim(ymin, ymax)
    
    
    plt.suptitle('Mean square error on each fold: coordinate descent')

Lasso/ENet path

In [ ]:
#normalize data?
X2=np.copy(X)
XT=X2.T
if normalizeFeatures:
    XT -= np.mean(XT, axis=0)

    XT /= XT.std(axis=0)

metric_alphas_lasso = {}
metric_coefs_lasso = {}
metric_alphas_enet = {}
metric_coefs_enet = {}

for metrique in metric_arrays:
    try:
        y=metric_coeffs[metrique]*metric_arrays[metrique]
    except KeyError as e:
        print(f'{metrique} not in metric_coeffs, lasso not performed')
        continue
        
    # Compute paths

    eps = 1e-1  # the smaller it is the longer is the path
    
    print(metrique)
    print("Computing regularization path using the lasso...")
    alphas_lasso, coefs_lasso, _ = lasso_path(XT, y, eps=eps, fit_intercept=True)

    metric_alphas_lasso[metrique]=alphas_lasso
    metric_coefs_lasso[metrique]=coefs_lasso
    
    print("Computing regularization path using the elastic net...")
    alphas_enet, coefs_enet, _ = enet_path(
        XT, y, eps=eps, l1_ratio=l1_ratio, fit_intercept=True)
    
    
    metric_alphas_enet[metrique]=alphas_enet
    metric_coefs_enet[metrique]=coefs_enet
    

    # Display results
    plt=pl
    plt.figure(1, figsize=(8,6))
    colors = cycle(['b', 'r', 'g', 'c', 'k'])
    neg_log_alphas_lasso = -np.log10(alphas_lasso)
    neg_log_alphas_enet = -np.log10(alphas_enet)
    for coef_l, coef_e, c in zip(coefs_lasso, coefs_enet, colors):
        l1 = plt.plot(neg_log_alphas_lasso, coef_l, c=c)
        l2 = plt.plot(neg_log_alphas_enet, coef_e, linestyle=':', c=c, alpha=0.30)
    
    plt.axvline(-np.log10(metric_alphaCV_lasso[metrique]), linestyle='-', color='k')
    plt.axvline(-np.log10(metric_alphaCV_enet[metrique]), linestyle=':', color='k')
        
        
    plt.xlabel('-Log(alpha)')
    plt.ylabel('coefficients')
    plt.title(f'Lasso and Elastic-Net Paths - {metrique}')
    plt.legend((l1[-1], l2[-1]), ('Lasso', 'Elastic-Net'), loc='lower left')
    plt.axis('tight')
    plt.show()


Lasso with custom alpha

In [ ]:
metric_alphaCV_lasso, metric_alphaCV_enet

In [ ]:
metric_alpha_lasso_custom = metric_alphaCV_lasso
metric_alpha_enet_custom = metric_alphaCV_enet

conservative=False
#more conservative
if conservative:
    metric_alpha_lasso_custom['deltV']=0.2
    metric_alpha_lasso_custom['deltC']=0.2
    metric_alpha_lasso_custom['propV']=0.25
    metric_alpha_lasso_custom['varco_V']=0.15
    metric_alpha_lasso_custom['varco_C']=metric_alpha_lasso_custom['varco_C']-0.01   
    metric_alpha_lasso_custom['nPVI_V']=0.2
    metric_alpha_lasso_custom['rPVI_C']=0.3
    
    metric_alpha_enet_custom['deltV']=0.6
    #metric_alpha_enet_custom['deltC']=0.2
    metric_alpha_enet_custom['propV']=0.8
    metric_alpha_enet_custom['varco_V']=0.7
    metric_alpha_enet_custom['varco_C']=metric_alpha_lasso_custom['varco_C']-0.01   
    metric_alpha_enet_custom['nPVI_V']=0.5
    metric_alpha_enet_custom['rPVI_C']=1
    


In [ ]:
metric_coeffs_lasso = {}  #double f, caution: a dictionary _coefs have been defined previously
metric_coeffs_enet = {}

metric_intercept_lasso = {} 
metric_intercept_enet = {}

def print_coeffs(intercept, coeffs):
    st_l=[str(intercept)]
    for ind in np.nonzero(coeffs)[0]:
        st_l.append(f' + ({coeffs[ind]:0.4f})*x[{ind}]')
    print(''.join(st_l))

for metrique in metric_arrays:
    
    y=metric_coeffs[metrique]*metric_arrays[metrique]
    
    model_lasso = Lasso(alpha=metric_alpha_lasso_custom[metrique], fit_intercept=True)
    model_lasso.fit(XT, y)
    
    metric_intercept_lasso[metrique] = model_lasso.intercept_/metric_coeffs[metrique]
    metric_coeffs_lasso[metrique] = model_lasso.coef_/metric_coeffs[metrique] #XXX multiplied again by normalization factors
    if normalizeFeatures:
        metric_coeffs_lasso[metrique] /= X.std(axis=1)
        
    print(f'Lasso {metrique} : {np.count_nonzero(model_lasso.coef_)} non-zero coefficients')
    #print_coeffs(model_lasso.intercept_, model_lasso.coef_)
    
    model_enet = ElasticNet(alpha=metric_alpha_enet_custom[metrique], l1_ratio=l1_ratio, fit_intercept=True)
    model_enet.fit(XT, y)
    
    metric_intercept_enet[metrique] = model_enet.intercept_/metric_coeffs[metrique]
    metric_coeffs_enet[metrique] = model_enet.coef_/metric_coeffs[metrique]
    if normalizeFeatures:
        metric_coeffs_enet[metrique]/=X.std(axis=1)
    
    print(f'ElasticNet {metrique} : {np.count_nonzero(model_enet.coef_)} non-zero coefficients')
    #print_coeffs(model_enet.intercept_, model_enet.coef_)
    
    
    print('')

Scatter plots

In [ ]:
for metrique in metric_arrays:
    for (coeffs, intercept, name) in [(metric_coeffs_lasso[metrique], metric_intercept_lasso[metrique], 'Lasso'),
                                    (metric_coeffs_enet[metrique], metric_intercept_enet[metrique], 'ElasticNet')]:
        c=[]
        met_arr=metric_arrays[metrique]
        for file in list_files:
            c.append(c_lang[d_lang[file]])
        y=np.dot(X.T, coeffs)
        y+=intercept

        #HACK for legend
        from matplotlib.lines import Line2D
        custom_lines = [Line2D([0], [0], c=f'C{i}') for i in range(len(c_lang))]

        pl.figure(figsize=(8,6))
        pl.scatter(met_arr, y, c=c)
        pl.xlabel(metrique)
        pl.legend(custom_lines, c_lang)
        pl.title(f'Regression with activations ({name}) for {metrique}')

Save coeffs to JSON file

In [ ]:

if saveCoeffs:
    coeffs_dic={}
    for metrique in metric_arrays:
        dic={}
        dic['mult_factor']=metric_coeffs[metrique]
        
        dic2={}
        dic2['alpha']=metric_alpha_lasso_custom[metrique]
        dic2['coeffs']=list(metric_coeffs_lasso[metrique])
        dic2['intercept']=metric_intercept_lasso[metrique]
        
        dic['lasso']=dic2
        
        
        dic2={}
        dic2['alpha']=metric_alpha_enet_custom[metrique]
        dic2['coeffs']=list(metric_coeffs_enet[metrique])
        dic2['intercept']=metric_intercept_enet[metrique]
        
        dic['enet']=dic2
        
        coeffs_dic[metrique]=dic
    #activations_name=activations_folder.split('/')[-1]
    json_filename=activations_name
    if conservative:
        json_filename+='_conservative'
    with open(f'./corpus_ramus/coeffs/{json_filename}.json', 'w') as f:
        json.dump(coeffs_dic, f, sort_keys=True, indent=4)